In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib as mpl 
#mpl.use("module://mplcairo.qt") 
from mplcairo import operator_t
from mpl_toolkits.axes_grid1 import make_axes_locatable
import subprocess
from lib.funcs import *

In [2]:
## Get the names and open data
renda_path = ('./data/CENSO/PessoaRenda_PB.csv') 

data = pd.read_csv(renda_path,                  
                sep=';', 
                encoding="utf-8",
                dtype={'Cod_setor': str,
                      'V020': float,
                      'V021': float,
                      'V022': float},
                   na_values = ['X'],
                lineterminator='\n')

In [3]:
## Open SHP
setores = gpd.read_file('./data/shp/jp_shp.shp')
setores = setores.to_crs(epsg=3857)

In [4]:
border = setores
border['new_column'] = 0
border = border.dissolve(by='new_column')
border = border[['geometry']]
border['geometry'] = border['geometry'].buffer(0.0001)

In [5]:
# Select only the data we are interested in
#v001 = Moradores em domicilios particular e domicilios coletivos
renda = data[['Cod_setor', 'V020', 'V021', 'V022']]
renda = renda.rename(columns={'V020': 'num_pessoas',
                             'V021': 'num_pessoas_rendimento',
                             'V022': 'rendimento_nominal'})

setores = setores[['geometry', 'NM_BAIRRO', 'CD_GEOCODI']]

In [6]:
populacao = pd.read_csv('./data/POPULACAO/populacao_parse.csv',
                       dtype={'Cod_setor': str})

In [7]:
domicilios = pd.read_csv('./data/DOMICILIOS/domicilios_parse.csv',
                       dtype={'Cod_setor': str})

In [8]:
renda = pd.merge(populacao, renda, on='Cod_setor')
renda = pd.merge(domicilios, renda, on='Cod_setor')

In [9]:
renda['num_pessoas'] = renda['MORADORES']

In [10]:
renda.drop(['MORADORES'], axis=1, inplace=True)

In [11]:
renda

,Cod_setor,n_domicilios,num_pessoas,num_pessoas_rendimento,rendimento_nominal
0,250010605000001,436,1528,712.0,304733.0
1,250010605000002,148,500,273.0,177732.0
2,250010605000003,295,889,484.0,310937.0
3,250010605000004,217,860,372.0,141471.0
4,250010605000005,199,667,326.0,143194.0
...,...,...,...,...,...
5543,251720905000004,103,345,183.0,81061.0
5544,251720905000005,310,1105,525.0,212375.0
5545,251740705000001,278,876,469.0,219933.0
5546,251740705000002,184,603,368.0,135068.0


In [12]:
# Set the same inde
renda.set_index('Cod_setor', inplace=True)
setores.set_index('CD_GEOCODI', inplace=True)

In [13]:
setores = setores.join(renda)

In [14]:
bairros = setores.dissolve(by='NM_BAIRRO', aggfunc='sum')

In [15]:
bairros_interesse = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar', 'Pedro Gondim', 'Estados']
setores_recorte = setores.loc[setores['NM_BAIRRO'].isin(bairros_interesse)]

In [16]:
def get_renda_por_domicilio(row):
    try:
        renda = row["rendimento_nominal"] / row["n_domicilios"]
    except (ValueError, ZeroDivisionError):
        return np.nan
    return renda

In [17]:
setores_recorte['renda_per_capita'] = setores_recorte.apply(get_renda_per_capita, axis=1)
setores_recorte['renda_por_trabalhador'] = setores_recorte.apply(get_renda_por_trabalhador, axis=1)
setores_recorte['renda_por_domicilio'] = setores_recorte.apply(get_renda_por_domicilio, axis=1)
bairros['renda_per_capita'] = bairros.apply(get_renda_per_capita, axis=1)
bairros['renda_por_trabalhador'] = bairros.apply(get_renda_por_trabalhador, axis=1)
bairros['renda_por_domicilio'] = bairros.apply(get_renda_por_domicilio, axis=1)

<ipython-input-17-0056966ddc1e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setores_recorte['renda_per_capita'] = setores_recorte.apply(get_renda_per_capita, axis=1)
<ipython-input-17-0056966ddc1e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setores_recorte['renda_por_trabalhador'] = setores_recorte.apply(get_renda_por_trabalhador, axis=1)
<ipython-input-17-0056966ddc1e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [18]:
bairros = bairros.round(2)
bairros

,geometry,n_domicilios,num_pessoas,num_pessoas_rendimento,rendimento_nominal,renda_per_capita,renda_por_trabalhador,renda_por_domicilio
NM_BAIRRO,,,,,,,,
Aeroclube,"POLYGON ((-3878845.787 -792159.924, -3878970.3...",3223.0,9649.0,5975.0,17493369.0,1812.97,2927.76,5427.67
Altiplano Cabo Branco,"POLYGON ((-3876916.727 -797181.774, -3877032.3...",1466.0,5233.0,2996.0,7316627.0,1398.17,2442.13,4990.88
Alto do Céu,"POLYGON ((-3880526.984 -791742.159, -3880526.7...",4651.0,16557.0,7110.0,4081383.0,246.50,574.03,877.53
Alto do Mateus,"POLYGON ((-3885968.582 -797638.000, -3886002.0...",4633.0,16281.0,7785.0,5279507.0,324.27,678.16,1139.54
Anatólia,"POLYGON ((-3878566.488 -797931.549, -3878554.7...",387.0,1162.0,761.0,1549911.0,1333.83,2036.68,4004.94
...,...,...,...,...,...,...,...,...
Trincheiras,"POLYGON ((-3883573.691 -795946.209, -3883573.6...",2061.0,6995.0,3421.0,2684683.0,383.80,784.77,1302.61
Valentina,"POLYGON ((-3878549.633 -804234.168, -3878556.9...",6318.0,22452.0,12371.0,11102453.0,494.50,897.46,1757.27
Varadouro,"POLYGON ((-3883870.618 -794980.219, -3883870.6...",1150.0,3720.0,1979.0,1427824.0,383.82,721.49,1241.59


In [19]:
bairros_recorte = bairros.loc[bairros.index.isin(bairros_interesse)]
bairros_recorte = bairros_recorte.drop('geometry', axis=1)
bairros_recorte = pd.DataFrame(bairros_recorte)

In [20]:
setores_recorte.to_file('./data/RENDA/setores_recorte.shp')
bairros.to_file('./data/RENDA/bairros.shp')

bairros_recorte.to_csv('./OUTPUT/CSV/bairros_recorte_populacao_e_renda.csv', decimal='.', sep=',', float_format='%.0f')

In [ ]:
export(setores_recorte, 'renda_setores_recorte', 3857)
export(bairros, 'renda_bairros', 3857)